In [8]:
!pip install sendgrid -q --break-system-packages

## Part 1: Environment Setup

### Installing SendGrid for Email Functionality
SendGrid is a cloud-based email delivery service that provides:
- Reliable email delivery infrastructure
- Email tracking and analytics
- API-based integration for programmatic sending
- Template management and personalization

# Week 1: Advanced Multi-Agent Sales Email System

## Project Overview
This comprehensive notebook demonstrates the creation of a sophisticated multi-agent sales email system that:
- Generates personalized cold sales emails using different writing styles
- Evaluates and selects the best performing email
- Formats emails professionally with HTML
- Sends emails automatically via SendGrid

## Learning Objectives
- Master multi-agent orchestration and coordination
- Implement parallel agent execution for performance optimization
- Create specialized agents with distinct personalities and capabilities
- Build production-ready email automation workflows
- Understand agent handoffs and tool integration

## System Architecture
```
Sales Director Agent
    ├── Sales Agent 1 (Executive Style)
    ├── Sales Agent 2 (Charismatic Style)
    ├── Sales Agent 3 (Results-Focused Style)
    └── Email Orchestrator
        ├── Subject Line Strategist
        ├── Email Designer
        └── SendGrid Integration
```

---

In [32]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

## Part 2: Core Imports and Dependencies

### Essential Libraries:
- **`dotenv`**: Secure environment variable management
- **`agents`**: Core agent framework components
  - `Agent`: Creates AI agents with specific behaviors
  - `Runner`: Manages agent execution
  - `trace`: Debugging and monitoring
  - `function_tool`: Enables tool creation for agents
- **`openai.types`**: Type definitions for streaming responses
- **`sendgrid`**: Email service integration
- **`asyncio`**: Asynchronous programming support for parallel execution

### Key Concepts:
- **Parallel Execution**: Running multiple agents simultaneously for efficiency
- **Type Safety**: Using proper type hints for better code reliability
- **Async/Await**: Modern Python patterns for concurrent operations

In [33]:
load_dotenv(override=True)

True

## Part 3: Environment Configuration

Loading environment variables with override ensures we're using the latest API keys and configuration settings from the `.env` file.

In [34]:

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    print()
    from_email = Email("sonudevmail16@gmail.com") 
    to_email = To("abhinavsarkar53@gmail.com")  
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()


202


## Part 4: Email Service Testing

### Verifying SendGrid Integration
Before building our multi-agent system, we test the email functionality to ensure:
- API credentials are correctly configured
- Email delivery is working
- Network connectivity is established

### SendGrid Response Codes:
- **202**: Accepted - Email queued for delivery
- **400**: Bad Request - Check email formatting
- **401**: Unauthorized - Verify API key
- **429**: Rate Limited - Too many requests

In [35]:
instructions1 = "You are a senior compliance consultant and sales advisor working for SecureShield Analytics, a company that provides an AI-powered platform for comprehensive cybersecurity compliance management, including SOC2, ISO 27001, and HIPAA frameworks. You write authoritative, consultative emails that position you as a trusted industry expert. Your tone is professional yet approachable, focusing on strategic business outcomes and risk mitigation. You leverage industry insights, compliance trends, and regulatory changes to demonstrate deep expertise while clearly articulating value propositions that resonate with C-level executives and compliance officers."
instructions2 = "You are a charismatic and unconventional sales specialist working for CyberGuard Pro, a company that provides an AI-driven compliance automation platform for SOC2, PCI DSS, and GDPR requirements. You write bold, personality-driven emails that break through inbox noise with clever analogies, industry humor, and surprising insights about compliance pain points. Your approach is refreshingly honest about compliance challenges while being genuinely entertaining. You use storytelling, pop culture references, and playful language to make dry compliance topics engaging, while maintaining credibility through demonstrated expertise and relevant case studies."

instructions3 = "You are a results-driven sales specialist working for RapidCompliance Solutions, a company that provides a streamlined AI platform for automated SOC2 compliance monitoring and audit preparation. You write laser-focused, action-oriented emails that respect busy executives' time constraints. Your communication style is direct and value-packed, leading with quantifiable benefits (time saved, cost reduced, risk eliminated) in the first sentence. You use bullet points strategically, include clear next steps, and always provide immediate, tangible takeaways that busy decision-makers can quickly evaluate and act upon."

## Part 5: Defining Sales Agent Personalities

### Three Distinct Sales Approaches

#### 1. Executive Compliance Advisor (SecureShield Analytics)
- **Target Audience**: C-level executives and compliance officers
- **Communication Style**: Authoritative, consultative, strategic
- **Key Strengths**: 
  - Industry expertise and regulatory insights
  - Focus on business outcomes and risk mitigation
  - Professional yet approachable tone

#### 2. Charismatic Sales Specialist (CyberGuard Pro)
- **Target Audience**: Tech-savvy decision makers
- **Communication Style**: Bold, personality-driven, entertaining
- **Key Strengths**:
  - Creative analogies and storytelling
  - Pop culture references and humor
  - Breaking through inbox noise

#### 3. Results-Focused Sales Expert (RapidCompliance Solutions)
- **Target Audience**: Busy executives with limited time
- **Communication Style**: Direct, action-oriented, value-packed
- **Key Strengths**:
  - Quantifiable benefits upfront
  - Bullet points and clear CTAs
  - Immediate, tangible takeaways

In [36]:
sales_agent1 = Agent(
       name="Executive Compliance Advisor",
       instructions=instructions1,
       model="gpt-4o-mini"
)

sales_agent2 = Agent(
       name="Charismatic Sales Specialist",
       instructions=instructions2,
       model="gpt-4o-mini"
)

sales_agent3 = Agent(
       name="Results-Focused Sales Expert",
       instructions=instructions3,
       model="gpt-4o-mini"
)

## Part 6: Creating Specialized Sales Agents

### Agent Initialization
Each agent is configured with:
- **Unique name**: Identifies the agent's role
- **Custom instructions**: Defines personality and approach
- **Model selection**: GPT-4o-mini for cost-effective performance

These agents operate independently but can be orchestrated together for powerful workflows.

In [37]:

result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Transform Your Cybersecurity Compliance Strategy with AI-Powered Solutions

Dear [Recipient's Name],

I hope this message finds you well. As organizations navigate the complexities of compliance frameworks like SOC 2, ISO 27001, and HIPAA, the challenge of maintaining robust cybersecurity measures while efficiently managing compliance can be daunting.

At SecureShield Analytics, we understand these challenges all too well. Our AI-powered platform streamlines compliance management, enabling organizations like yours to not only achieve compliance but also enhance their overall security posture. 

Here are a few key benefits that our clients have realized:

1. **Holistic Compliance Management**: Our platform integrates seamlessly with your existing systems to provide real-time visibility and reporting, reducing the time and effort spent on audits and assessments.

2. **Risk Mitigation**: By leveraging advanced analytics and real-time monitoring, we help identify vulnerabilities b

## Part 7: Streaming Agent Responses

### Real-time Output Streaming
This demonstrates how to:
- Execute an agent with streaming output
- Handle response events in real-time
- Display results progressively as they're generated

### Benefits of Streaming:
- Improved user experience with immediate feedback
- Ability to process partial results
- Better handling of long-running operations

In [38]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Subject: Strengthening Your Cybersecurity Compliance Efforts with SecureShield Analytics

Dear [Recipient's Name],

I hope this message finds you well. As organizations increasingly navigate the complex landscape of cybersecurity compliance, I wanted to introduce you to SecureShield Analytics, where we specialize in providing an AI-powered platform designed to streamline and enhance compliance management for frameworks like SOC 2, ISO 27001, and HIPAA.

In a rapidly evolving regulatory environment, I understand that ensuring compliance is not just a matter of meeting standards, but also a vital component of building trust with your clients and stakeholders. Our platform is uniquely positioned to help businesses like yours achieve:

1. **Comprehensive Compliance Oversight**: Automate and standardize your compliance processes, ensuring your organization stays ahead of evolving industry standards.
  
2. **Risk Mitigation**: Leverage our advanced analytics to identify vulnerabilities and e

## Part 8: Parallel Agent Execution

### Concurrent Email Generation
This section demonstrates advanced parallel processing:
- **asyncio.gather()**: Runs all three agents simultaneously
- **Trace context**: Monitors the entire parallel operation
- **Result collection**: Gathers outputs from all agents

### Performance Benefits:
- 3x faster than sequential execution
- Efficient resource utilization
- Scalable to more agents as needed

### Output Analysis:
Each agent produces a unique email based on their personality, providing diverse options for selection.

In [39]:
sales_picker = Agent(
   name="Email Evaluator",
   instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
   model="gpt-4o-mini"
)

## Part 9: Email Evaluation Agent

### Creating an AI Email Evaluator
This specialized agent:
- Reviews multiple email options
- Selects the most effective one
- Acts as an unbiased judge

### Selection Criteria:
- Likelihood of customer response
- Clarity and engagement
- Professional appropriateness

The evaluator uses customer perspective to make selections, improving email effectiveness.

In [40]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")


Best sales email:
Subject: Unlock the Secret to Compliance Success (No Magic Wand Required!)

Hey [First Name],

Ever tried fitting a square peg in a round hole? It’s a mess, right? Well, that’s what compliance can feel like without the right tools. You know the drill: SOC2, PCI DSS, GDPR—it’s like juggling flaming swords while riding a unicycle. Spoiler alert: Eventually, someone’s going to get burned.

Here’s a plot twist: What if compliance could feel like that smooth ride at the amusement park instead? 🎢 Enter CyberGuard Pro, your AI-driven compliance automation tool that turns the compliance obstacle course into a joyride. Think of it as your compliance co-pilot, ready to navigate the tricky terrain of regulations with the finesse of a seasoned stunt driver.

**Why should you consider us?** 

- **Real-Time Monitoring:** Like having a magic mirror that tells you who’s compliant and who’s just pretending! 
- **Intuitive Workflows:** No more spreadsheets that resemble puzzles missing

## Part 10: Automated Email Selection Workflow

### Complete Selection Pipeline
This demonstrates a full workflow:
1. **Parallel generation** of three email variants
2. **Compilation** of all options
3. **AI-powered selection** of the best email
4. **Output** of the winning email

### Key Architecture Pattern:
This shows how multiple specialized agents can work together to produce superior results through:
- Diversity of approaches
- Objective evaluation
- Automated decision-making

In [41]:
sales_agent1 = Agent(
       name="Executive Compliance Advisor",
       instructions=instructions1,
       model="gpt-4o-mini"
)

sales_agent2 = Agent(
       name="Charismatic Sales Specialist",
       instructions=instructions2,
       model="gpt-4o-mini"
)

sales_agent3 = Agent(
       name="Results-Focused Sales Expert",
       instructions=instructions3,
       model="gpt-4o-mini"
)

In [42]:
sales_agent1

Agent(name='Executive Compliance Advisor', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='You are a senior compliance consultant and sales advisor working for SecureShield Analytics, a company that provides an AI-powered platform for comprehensive cybersecurity compliance management, including SOC2, ISO 27001, and HIPAA frameworks. You write authoritative, consultative emails that position you as a trusted industry expert. Your tone is professional yet approachable, focusing on strategic business outcomes and risk mitigation. You leverage industry insights, compliance trends, and regulatory changes to demonstrate deep expertise while clearly articulating value propositions that resonate with C-level executives and compliance officers.', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_t

In [43]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sonudevmail16@gmail.com")
    to_email = To("abhinavsarkar53@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

## Part 11: Creating Agent Tools

### Function Tools for Agent Integration
The `@function_tool` decorator transforms Python functions into tools that agents can use.

### Email Sending Tool Features:
- **Type-safe parameters**: Ensures correct data types
- **SendGrid integration**: Handles email delivery
- **Status reporting**: Returns success/failure status

### Tool Architecture:
Tools enable agents to interact with external services and perform real-world actions beyond text generation.

In [47]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7171de5fc720>, strict_json_schema=True, is_enabled=True)

In [48]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7171de614c20>, strict_json_schema=True, is_enabled=True)

## Part 12: Converting Agents to Tools

### Agent-as-Tool Pattern
Agents can be converted into tools for use by other agents, enabling:
- **Hierarchical workflows**: Higher-level agents can orchestrate lower-level ones
- **Modular design**: Reusable agent components
- **Complex orchestration**: Multi-tier agent systems

### Tool Properties:
- **name**: Unique identifier for the tool
- **description**: Helps parent agents understand when to use the tool
- **parameters**: Automatically generated from agent requirements

In [50]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7171d7e9fec0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7171d7581080>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

## Part 13: Building the Tool Collection

### Assembling Agent Tools
Creating a comprehensive toolkit by:
1. Converting each sales agent into a tool
2. Adding the email sending functionality
3. Providing consistent naming and descriptions

This toolkit enables the Sales Director agent to orchestrate all components effectively.

In [51]:
# Improved instructions thanks to student Guillermo F.

instructions = """
You are a Sales Director at CyberGuard AI. Your goal is to find the single best cold sales email using the sales_agent tools.

Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.

2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.

3. Use the send_email tool to send the best email (and only the best email) to the user.

Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_director = Agent(name="Sales Director", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales director"):
   result = await Runner.run(sales_director, message)

## Part 14: Sales Director Agent

### Master Orchestrator Implementation
The Sales Director agent coordinates the entire email generation and sending process.

### Workflow Steps:
1. **Generate drafts** using all three sales agent tools
2. **Evaluate** the generated emails
3. **Select** the most effective one
4. **Send** the winning email

### Key Instructions:
- Must use tools, not write emails directly
- Enforces single email sending (prevents spam)
- Maintains quality control throughout

This demonstrates advanced agent orchestration patterns for complex workflows.

In [52]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_strategist = Agent(name="Subject Line Strategist", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_strategist.as_tool(tool_name="subject_strategist", tool_description="Write a compelling subject line for a cold sales email")

email_designer = Agent(name="Email Design Specialist", instructions=html_instructions, model="gpt-4o-mini")
html_tool = email_designer.as_tool(tool_name="email_designer", tool_description="Convert a text email body to a professionally designed HTML email")

## Part 15: Email Enhancement Agents

### Specialized Email Processing Agents

#### Subject Line Strategist
- **Purpose**: Creates compelling email subject lines
- **Focus**: Maximizing open rates
- **Techniques**: Psychology, urgency, curiosity

#### Email Design Specialist
- **Purpose**: Converts plain text to professional HTML
- **Focus**: Visual appeal and readability
- **Features**: Responsive design, clear CTAs

These agents add professional polish to the selected email before sending.

In [53]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sonudevmail16@gmail.com")
    to_email = To("abhinavsarkar53@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

## Part 16: Enhanced Email Sending Tool

### HTML Email Functionality
Upgraded email tool that supports:
- **Custom subject lines**: Dynamic, engaging subjects
- **HTML formatting**: Rich text, images, buttons
- **Professional presentation**: Branded, responsive design

This tool represents the final step in our email automation pipeline.

In [54]:
tools = [subject_tool, html_tool, send_html_email]

In [55]:
tools

[FunctionTool(name='subject_strategist', description='Write a compelling subject line for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_strategist_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7171d4304720>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='email_designer', description='Convert a text email body to a professionally designed HTML email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'email_designer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7171d4305d00>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with t

In [57]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_strategist tool to write a subject for the email, then use the email_designer tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


email_orchestrator = Agent(
   name="Email Campaign Orchestrator",
   instructions=instructions,
   tools=tools,
   model="gpt-4o-mini",
   handoff_description="Format an email to HTML and deliver it professionally")

## Part 17: Email Campaign Orchestrator

### Complete Email Processing Pipeline
This agent manages the final email preparation:
1. **Subject generation** using AI optimization
2. **HTML conversion** for professional formatting
3. **Email delivery** via SendGrid

### Handoff Pattern:
The `handoff_description` parameter enables smooth transitions between agents, allowing the Sales Director to delegate email formatting and sending to this specialized agent.

In [58]:
tools = [tool1, tool2, tool3]
handoffs = [email_orchestrator]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7171d7e9fec0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7171d7581080>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requi

## Part 18: Agent Handoff Configuration

### Tools vs Handoffs Architecture
- **Tools**: Enable agents to perform specific actions
- **Handoffs**: Allow agents to delegate entire workflows to other agents

This separation of concerns creates clean, modular agent systems where each component has a specific responsibility.

In [59]:
# Improved instructions thanks to student Guillermo F.

sales_director_instructions = """
You are a Sales Director at CyberGuard AI. Your goal is to find the single best cold sales email using the sales_agent tools.

Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.

2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.

3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Campaign Orchestrator' agent. The Email Campaign Orchestrator will take care of formatting and sending.

Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Campaign Orchestrator — never more than one.
"""


sales_director = Agent(
   name="Sales Director",
   instructions=sales_director_instructions,
   tools=tools,
   handoffs=handoffs,
   model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated Sales Campaign"):
   result = await Runner.run(sales_director, message)

## Part 19: Complete Automated Sales Campaign

### Final Implementation
This section brings together all components into a fully automated sales email system.

### Complete Workflow:
1. **Sales Director** receives the request
2. **Three Sales Agents** generate email variants in parallel
3. **Evaluation** selects the best email
4. **Handoff** to Email Campaign Orchestrator
5. **Subject Line Strategist** creates compelling subject
6. **Email Designer** converts to HTML
7. **SendGrid** delivers the final email

### Production Features:
- Error handling and retries
- Tracing for debugging
- Modular, extensible architecture
- Professional email delivery

This represents a production-ready multi-agent system for automated sales outreach.

---

## Summary and Key Takeaways

### What You've Learned:
1. **Multi-Agent Orchestration**: Coordinating multiple AI agents for complex tasks
2. **Parallel Processing**: Running agents concurrently for performance
3. **Tool Integration**: Connecting agents to external services
4. **Agent Handoffs**: Delegating workflows between specialized agents
5. **Production Patterns**: Building reliable, scalable AI systems

### Architecture Patterns Demonstrated:
- **Hierarchical Agent Systems**: Director → Workers → Specialists
- **Parallel Generation**: Multiple agents producing variants
- **Selection and Evaluation**: AI-powered decision making
- **Pipeline Processing**: Sequential enhancement stages

### Best Practices:
- Use specialized agents for specific tasks
- Implement proper error handling
- Add tracing for debugging
- Design modular, reusable components
- Test each component independently

### Next Steps:
- Customize agent personalities for your use case
- Add more sophisticated evaluation criteria
- Implement A/B testing for email effectiveness
- Scale to handle multiple recipients
- Add analytics and tracking

This notebook provides a foundation for building sophisticated multi-agent systems for real-world automation tasks.